In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
from glob import glob

import math
import random 

import cv2                                 # image를 읽기 위한 open cv library
import xml.etree.ElementTree as et         # xml 파일을 parsing 하기 위한 library
from matplotlib.patches import Rectangle   # Bounding box를 그리기 위함

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.applications import EfficientNetB0

import albumentations as A    # CoarseDropout 인지 안 된다는 문제 발생 -> pip install 아래 세 가지 코드 실행 필요

import tensorflow_hub as hub

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

drive_project_root = 'drive/MyDrive/data/'
sys.path.append(drive_project_root)

image_root = 'drive/MyDrive/data/images/'
anno_root = 'drive/MyDrive/data/annotations/'

image_dir = image_root
bbox_dir = anno_root + 'xmls/'    # bounding box
seg_dir = anno_root + 'trimaps/'  # segmentation map

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

!ls

In [ ]:
pip install git+https://github.com/albumentations-team/albumentations.git  # Data Agumentation

In [ ]:
pip uninstall opencv-python

In [ ]:
pip install opencv-python

---
# Transfer Learning
- 배경
  - dataset의 고양이와 개 비율 = 2:5
    - 모든 data를 강아지로 찍어도 성능이 70% 정도
  - 앞에서 만든 모델의 성능 : validation data 70% accuracy
  - 즉, 모델 학습을 했음에도 불구하고 찍었을 때와 큰 차이가 없음
  - 그냥 학습을 했을 때와 data augmentation 적용했을 때도 최종 모델 성능 차이가 거의 없었음
  - 모델 바꿀 필요 있음 -> Transfer Learning
- Transfer Learning
  - 특정 분야에서 학습된 모델을 유사하거나 전혀 새로운 분야에 사용하는 것
    - 수십만 장의 image 학습하고 유의미한 feature 추출
    - 이를 oxford pet data 분류에 사용
  - 학습데이터가 적을 때 유용 + 학습 속도, 최종 결과도 모두 좋음 -> 항상 모델을 pre train 하는 것 추천
- pre trained된 모델을 가져올 수 있는 곳
  - https://github.com/keras-team/keras-applications
  - https://keras.io/api/applications/
  - VGG, ResNet, EfficientNet, etc.
  - 이 중 image net 분류에 대해 성능(Top-1 accuracy 부분)이 좋게 적혀 있는 모델일수록 내가 하려는 task에 쓸 때도 성능이 좋음
  - 주의 : 모델의 Size가 클수록 학습속도 느림
    - 느려도 상관 없는지 vs real-time이 중요한지
    - accuracy와 size의 trade off 잘 고려해서 선택할 것

---
# 1. EfficientNet
- 모델을 불러서 내가 사용하려는 것에 맞게 변환하기
- `model.summary()` 결과
  - efficientnetb0가 하나의 layer처럼 사용됨
  - 앞에서 쓴 모델은 parameter 수가 27만 개 정도 됐는데 parameter 수도 훨씬 많아짐 (400만)
  - 마지막 dense : 실제 분류하는 layer

In [ ]:
def get_model(input_shape):

    inputs = keras.Input(input_shape)

    # Feature extract
    base_model = EfficientNetB0(
        input_shape=input_shape,

        # 만약 아무 것도 안 쓰면 transfer을 사용하는 게 아니라 모델 구현체만 가져다 학습에 사용하겠다는 의미
        weights='imagenet',   # imagenet의 pre trained 된 weight값 사용
        
        # github 링크에서 모델 코드 보면 if include_top: 부분이 있음
        # whether to include the fully-connected layer at the top of the network
        # 해당 모델이 image를 분류하기 위해 사용한 layer들
        # = image에서 feature를 추출하는 layer 외에 그 feature를 가지고 분류하는 layer 부분임
        # = 내가 사용하려는 task랑 크게 관련은 없음 + 무거움
        # -> 이걸 가져다 쓰지 않고 (False) 내 task에 맞게 따로 구현하고자 함
        include_top=False,

        pooling='avg'  # Gloabl average pooling
    )

    x = base_model(inputs)  # image의 feature 추출
    outputs = layers.Dense(1, activation='sigmoid')(x)  # 이진분류
    model = keras.Model(inputs, outputs)

    return model

input_shape = (256, 256, 3)
model = get_model(input_shape)

# Transfer learning 할 때는 보통 사용하는 learning rate(0.001)보다 작은 값 사용
adam = keras.optimizers.Adam(lr=0.0001)

model.compile(
    optimizer=adam,
    loss='binary_crossentropy',
    metrics='accuracy'
)

model.summary()

In [ ]:
class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(
                    p=0.5,
                    shift_limit=0.05,  # 이미지의 가로 길이가 최대 몇% 넘어가도 되는 지
                    scale_limit=0.05,  # 이미지를 최대 몇% 확대/축소할지
                    rotate_limit=15,
                ),

                # 이미지에 구멍을 뚫는 것
                A.CoarseDropout(
                    p=0.5,
                    max_holes=8,                 # 최대 구멍 개수
                    max_height=int(0.1 * size),  # 가로 최대 길이 : 이미지의 10%
                    max_width=int(0.1 * size),
                ),

                A.RandomBrightnessContrast(p=0.2),
            ])

    def __call__(self, **kwargs):
        if self.transform:   # train mode인 경우
            augmented = self.transform(**kwargs)
            img = augmented['image']
            return img

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, image_size,
                 fold, mode='train', shuffle=True):
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.fold = fold
        self.mode = mode
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
        
        #### Remove invalid files
        #### https://github.com/tensorflow/models/issues/3134
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]
        self.df = self.df[~self.df['file_name']. \
                          isin(invalid_filenames)]

        self.transform = Augmentation(image_size, mode)

        self.on_epoch_end()
            
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
        
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        
        return np.array(batch_x), np.array(batch_y)
        
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for _, r in data.iterrows():
            file_name = r['file_name']
            
            image = cv2.imread(f'drive/MyDrive/data/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            image = cv2.resize(image, (self.image_size, self.image_size))

            if self.mode == 'train':
                image = image.astype('uint8')
                image = self.transform(image=image)

            image = image.astype('float32')
            image = image / 255.
            
            label = int(r['species']) - 1
            
            batch_x.append(image)
            batch_y.append(label)
        
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
csv_path = drive_project_root+'kfolds.csv'

train_generator =  DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

valid_generator =  DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

- 성능 굉장히 좋아짐

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1
)

In [ ]:
history = history.history

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_accuracy'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title("Accuracy")
plt.show()

---
# 2. TF Hub

- 배경
  - keras applications는 마지막 update가 몇 년 전
  - efficientNet 이후에 나온 모델들에 대해서는 update가 안 됨
- TF Hub
  - https://tfhub.dev/
  - tensorflow에서 공식적으로 운영
  - 최신 모델 (weight 포함) 지속 update 됨
  - keras에는 image만 있는 것과 달리 text, video, audio 모델까지 포함
- 방법
  - problem domains에서 image 클릭
  - filter 적용 : TF2 (version) + Finetunable 켜기
  - efficientNet v2 검색
    - efficientNet 이후에 나온 것
    - 이걸 사용해서 학습하면 많이 update가 되었기 때문에 편리함
  - 원하는 것 클릭 : imagenet/efficientnet_v2_imagenet1k_b0/feature_vector
    - 기존 모델들보다 성능 좋음
    - Usage 부분에 사용방법 코드 있음
    - 그대로 복사해서 가져오기

- 가장 최신 모델이라고 할 수 있는 efficientnet_v2를 tensorflow hub에서 가져와서 내 task 학습에 적용

In [ ]:
# Usage 코드
model = tf.keras.Sequential([
    hub.KerasLayer(
        "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2",  # 기존 구현된 모델
        trainable=True    # base 모델도 학습이 가능하게 설정
    ), 
    tf.keras.layers.Dense(1, activation='sigmoid')  # 내 모델의 (binary) classification layer 붙이기
])

model.build([None, 256, 256, 3])  # Batch input shape

adam = keras.optimizers.Adam(lr=0.0001)

model.compile(
    optimizer=adam,
    loss='binary_crossentropy',
    metrics='accuracy'
)

model.summary()

- 모델 사이즈가 커졌기 때문에 batch size를 절반 정도로 줄임

In [ ]:
csv_path = drive_project_root+'kfolds.csv'

train_generator =  DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=64,
    image_size=256,
    shuffle=True
)

valid_generator =  DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=64,
    image_size=256,
    shuffle=True
)

- efficientNet v2(TF Hub)가 b0과 큰 차이가 없는 이유
  - 지금 우리가 사용하는 task가 비교적 쉽기 때문
  - 다른 이미지 분류에 사용하면 b0보다 v2가 성능이 좋음

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1
)

In [ ]:
history = history.history

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_accuracy'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title("Accuracy")
plt.show()